парсинг репортов для заполнения таблицы с межфазными габаритами

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from math import sqrt

In [129]:
# пути 
workdir = Path(r'D:\work\2022_m\220_man-van\test\phases')

### часть 1 - расстояния в середине пролета

In [13]:
# read names
phase_tab = pd.read_excel(workdir / 'phases_dist.xlsx', sheet_name='c1')

In [14]:
phase_tab

,span_start,span_end,span_len
0,ПС_Мангазея,1,40.24
1,1,2,106.61
2,2,3,187.55
3,3,4,135.84
4,4,5,147.95
...,...,...,...
603,603,604,179.01
604,604,605,121.73
605,605,606,56.54
606,606,607,66.35


In [15]:
# calc clearance norm

# добавляем колонки 
phase_tab.assign(norma=np.nan)

for span in range(len(phase_tab)):
#for span in range(3):
    idx = phase_tab.iloc[span].name
    span_len = phase_tab.iloc[span, 2]

    if span_len <= 100:
        norma = 2
    elif 100 < span_len <= 150:
        norma = (span_len - 100)*1.2/50 + 2
    elif 150 < span_len <= 200:
        norma = (span_len - 150)*0.8/50 + 3.2
    elif 200 < span_len <= 300:
        norma = (span_len - 200)*1.5/100 + 4
    elif 300 < span_len <= 400:
        norma = (span_len - 300)*1.5/100 + 5.5
    elif 400 < span_len <= 500:
        norma = (span_len - 400)*1.5/100 + 7
    elif 500 < span_len <= 600:
        norma = (span_len - 500)*1.5/100 + 8.5
    else:
        norma = 10

    print(span_len, round(norma, 1))

    phase_tab.loc[idx, 'norma'] = round(norma, 1)


40.24 2
106.61 2.2
187.55 3.8
135.84 2.9
147.95 3.2
138.08 2.9
244.23 4.7
140.96 3.0
70.07 2
379.14 6.7
309.85 5.6
349.81 6.2
329.85 5.9
329.88 5.9
329.81 5.9
299.88 5.5
319.91 5.8
329.93 5.9
334.93 6.0
337.36 6.1
383.06 6.7
374.37 6.6
287.5 5.3
267.78 5.0
302.1 5.5
353.86 6.3
337.89 6.1
332.85 6.0
329.67 5.9
367.94 6.5
395.84 6.9
389.82 6.8
259.19 4.9
259.94 4.9
329.86 5.9
299.86 5.5
299.88 5.5
309.8 5.6
334.89 6.0
324.3 5.9
334.83 6.0
256.93 4.9
273.54 5.1
295.03 5.4
320.21 5.8
344.84 6.2
328.9 5.9
344.88 6.2
329.92 5.9
344.86 6.2
259.81 4.9
295.51 5.4
334.45 6.0
354.0 6.3
339.57 6.1
300.23 5.5
344.85 6.2
344.74 6.2
339.88 6.1
329.93 5.9
309.88 5.6
252.56 4.8
240.83 4.6
264.96 5.0
379.92 6.7
379.92 6.7
332.21 6.0
231.72 4.5
392.13 6.9
341.12 6.1
167.17 3.5
254.39 4.8
200.04 4.0
200.01 4.0
294.9 5.4
277.5 5.2
307.27 5.6
264.84 5.0
264.94 5.0
274.95 5.1
280.0 5.2
284.78 5.3
284.91 5.3
284.78 5.3
300.03 5.5
254.65 4.8
254.96 4.8
259.96 4.9
259.93 4.9
258.84 4.9
296.02 5.4
279.97 5.2
259

In [16]:
phase_tab

,span_start,span_end,span_len,norma
0,ПС_Мангазея,1,40.24,2.0
1,1,2,106.61,2.2
2,2,3,187.55,3.8
3,3,4,135.84,2.9
4,4,5,147.95,3.2
...,...,...,...,...
603,603,604,179.01,3.7
604,604,605,121.73,2.5
605,605,606,56.54,2.0
606,606,607,66.35,2.0


In [26]:
# read phases clearances
clearance_tab = pd.read_excel(workdir / 'phases_dist.xlsx', sheet_name='wir2wir_2c')

In [27]:
clearance_tab

,Row #,Source Start Structure,Source End Structure,Source Set,Source Phase,Source Wind From,Target Start Structure,Target End Structure,Target Set,Target Phase,Target Wind From,Station At Min. (m),Source Sag At Min. (m),Target Sag At Min. (m),Required Distance At Min. (m),Minimum Distance Total (m),Minimum Distance Horz. Comp. (m),Minimum Distance Vert. Comp. (m),Comment
0,1,ПС_Мангазея,1,14,1,Left,ПС_Мангазея,1,15,1,Left,3.46,0.00,0.06,1,3.758,3.520,-1.316,NaN
1,2,ПС_Мангазея,1,14,1,Left,ПС_Мангазея,1,16,1,Left,23.64,0.84,0.82,1,5.819,4.373,-3.839,NaN
2,3,ПС_Мангазея,1,14,1,Left,1,2,14,1,Left,38.46,0.00,0.00,1,7.143,7.127,-0.472,NaN
3,4,ПС_Мангазея,1,14,1,Left,1,2,15,1,Left,38.46,0.00,0.00,1,10.176,7.450,-6.932,NaN
4,5,ПС_Мангазея,1,15,1,Left,ПС_Мангазея,1,16,1,Left,5.67,0.24,0.33,1,3.567,3.436,0.957,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5516,5517,606,607,16,1,Left,607,ÏÑ_Âàíêîð,15,1,Left,166869.19,0.00,0.00,1,10.634,7.875,7.146,NaN
5517,5518,606,607,16,1,Left,607,ÏÑ_Âàíêîð,16,1,Left,166869.19,0.00,0.00,1,7.244,7.133,1.260,NaN
5518,5519,607,ÏÑ_Âàíêîð,14,1,Left,607,ÏÑ_Âàíêîð,15,1,Left,166891.75,0.40,0.45,1,2.698,1.979,-1.834,NaN
5519,5520,607,ÏÑ_Âàíêîð,14,1,Left,607,ÏÑ_Âàíêîð,16,1,Left,166892.22,0.38,0.42,1,6.492,5.553,-3.362,NaN


In [28]:
# добавляем колонки 
phase_tab.assign(mindist=np.nan, phnames=np.nan)

# идем по таблице 
for span in range(len(phase_tab)):
#for span in range(3):
    idx = phase_tab.iloc[span].name
    span_start = phase_tab.iloc[span, 0]
    select = clearance_tab.loc[clearance_tab['Source Start Structure '] == span_start].loc[clearance_tab['Target Start Structure '] == span_start][['Source Set  ', 'Target Set  ', 'Minimum Distance Total (m)']]
    
    if len(select) > 0:
        select2 = select[select['Minimum Distance Total (m)'] == min(select['Minimum Distance Total (m)'])]
        #print(span_start, str(select2.iloc[0, 0]) + ' - ' + str(select2.iloc[0, 1]) + ', ' + str(select2.iloc[0, 2]))
        phase_tab.loc[idx, 'mindist'] = select2.iloc[0, 2]
        phase_tab.loc[idx, 'phnames'] = str(select2.iloc[0, 0]) + ' - ' + str(select2.iloc[0, 1])
    else:
        print(span_start)


In [29]:
phase_tab

,span_start,span_end,span_len,norma,mindist,phnames,mindist_ew,phtoew,ewvert
0,ПС_Мангазея,1,40.24,2.0,3.567,15 - 16,6.153,17 - 12,5.670
1,1,2,106.61,2.2,6.482,14 - 15,8.376,17 - 13,7.997
2,2,3,187.55,3.8,6.196,4 - 5,7.944,7 - 3,7.618
3,3,4,135.84,2.9,6.185,15 - 16,8.437,17 - 13,8.129
4,4,5,147.95,3.2,6.418,15 - 16,8.674,18 - 13,8.181
...,...,...,...,...,...,...,...,...,...
603,603,604,179.01,3.7,6.250,4 - 5,8.727,8 - 3,7.521
604,604,605,121.73,2.5,5.746,14 - 15,NaN,NaN,NaN
605,605,606,56.54,2.0,6.603,15 - 16,8.308,17 - 13,7.985
606,606,607,66.35,2.0,6.419,15 - 16,8.702,17 - 13,8.448


In [48]:
clearance_tab.columns

Index(['Row #', 'Source Start Structure ', 'Source End Structure ',
       'Source Set  ', 'Source Phase  ', 'Source Wind From ',
       'Target Start Structure ', 'Target End Structure ', 'Target Set  ',
       'Target Phase  ', 'Target Wind From ', 'Station At Min.  (m)',
       'Source Sag At Min. (m)', 'Target Sag At Min. (m)',
       'Required Distance At Min. (m)', 'Minimum Distance Total (m)',
       'Minimum Distance Horz. Comp. (m)', 'Minimum Distance Vert. Comp. (m)',
       'Comment'],
      dtype='object')

In [31]:
# read ew clearances
clearance_tab = pd.read_excel(workdir / 'phases_dist.xlsx', sheet_name='ew_2c')

In [32]:
clearance_tab

,Row #,Source Start Structure,Source End Structure,Source Set,Source Phase,Source Wind From,Target Start Structure,Target End Structure,Target Set,Target Phase,Target Wind From,Station At Min. (m),Source Sag At Min. (m),Target Sag At Min. (m),Required Distance At Min. (m),Minimum Distance Total (m),Minimum Distance Horz. Comp. (m),Minimum Distance Vert. Comp. (m),Comment
0,1,ПС_Мангазея,1,17,1,Left,ПС_Мангазея,1,15,1,Left,36.82,0.52,0.00,1,10.829,7.351,7.951,NaN
1,2,ПС_Мангазея,1,17,1,Left,1,2,16,1,Left,39.11,0.00,0.00,1,10.394,6.509,8.103,NaN
2,3,ПС_Мангазея,1,18,1,Left,ПС_Мангазея,1,14,1,Left,2.85,0.49,0.00,1,8.099,3.686,7.211,NaN
3,4,ПС_Мангазея,1,18,1,Left,ПС_Мангазея,1,15,1,Left,25.66,2.22,0.70,1,7.630,4.932,5.821,NaN
4,5,ПС_Мангазея,1,18,1,Left,ПС_Мангазея,1,16,1,Left,14.60,2.13,0.79,1,11.971,8.934,7.968,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933,1934,606,607,17,1,Left,606,607,16,1,Left,166807.29,0.00,0.00,1,10.250,5.573,8.603,NaN
1934,1935,606,607,17,1,Left,607,ÏÑ_Âàíêîð,16,1,Left,166872.55,0.00,0.00,1,12.182,6.987,9.979,NaN
1935,1936,606,607,18,1,Left,605,606,16,1,Left,166805.68,0.00,0.00,1,9.583,4.209,8.609,NaN
1936,1937,606,607,18,1,Left,606,607,16,1,Left,166807.38,0.13,0.00,1,8.939,2.778,8.496,NaN


In [51]:
clearance_tab.columns

Index(['Row #', 'Source Start Structure ', 'Source End Structure ',
       'Source Set  ', 'Source Phase  ', 'Source Wind From ',
       'Target Start Structure ', 'Target End Structure ', 'Target Set  ',
       'Target Phase  ', 'Target Wind From ', 'Station At Min.  (m)',
       'Source Sag At Min. (m)', 'Target Sag At Min. (m)',
       'Required Distance At Min. (m)', 'Minimum Distance Total (m)',
       'Minimum Distance Horz. Comp. (m)', 'Minimum Distance Vert. Comp. (m)',
       'Comment'],
      dtype='object')

In [33]:
# calc ew 3d dist

# добавляем колонки 
phase_tab.assign(mindist_ew=np.nan, phtoew=np.nan, ewvert=np.nan,)

# идем по таблице 
for span in range(len(phase_tab)):
#for span in range(3):
    idx = phase_tab.iloc[span].name
    span_start = phase_tab.iloc[span, 0]
    select = clearance_tab.loc[clearance_tab['Source Start Structure '] == span_start].loc[clearance_tab['Target Start Structure '] == span_start][['Source Set  ', 'Target Set  ', 'Minimum Distance Total (m)', 'Minimum Distance Vert. Comp. (m)']]
    #print(select)
    if len(select) > 0:
        select2 = select[select['Minimum Distance Total (m)'] == min(select['Minimum Distance Total (m)'])]
      
        #print(select2)
        phase_tab.loc[idx, 'mindist_ew'] = select2.iloc[0, 2]
        phase_tab.loc[idx, 'phtoew'] = str(select2.iloc[0, 0]) + ' - ' + str(select2.iloc[0, 1])
        phase_tab.loc[idx, 'ewvert'] = select2.iloc[0, 3]






In [34]:
phase_tab

,span_start,span_end,span_len,norma,mindist,phnames,mindist_ew,phtoew,ewvert
0,ПС_Мангазея,1,40.24,2.0,3.567,15 - 16,7.630,18 - 15,5.821
1,1,2,106.61,2.2,6.482,14 - 15,8.614,18 - 16,8.130
2,2,3,187.55,3.8,6.196,4 - 5,8.602,8 - 6,8.051
3,3,4,135.84,2.9,6.185,15 - 16,8.628,18 - 16,8.063
4,4,5,147.95,3.2,6.418,15 - 16,8.563,17 - 16,8.025
...,...,...,...,...,...,...,...,...,...
603,603,604,179.01,3.7,6.250,4 - 5,7.911,8 - 6,7.300
604,604,605,121.73,2.5,5.746,14 - 15,NaN,NaN,NaN
605,605,606,56.54,2.0,6.603,15 - 16,8.187,18 - 16,7.610
606,606,607,66.35,2.0,6.419,15 - 16,8.939,18 - 16,8.496


In [35]:
# save tab
phase_tab.to_csv(workdir / 'phases_c2.txt', sep='\t', encoding = 'cp1251')

### теперь найдем расстояния между тк на опоре

In [130]:
# read id 

phase2_tab = pd.read_excel(workdir / 'clearance_grd.xlsx', sheet_name='str_names', names=['id'], usecols='A')

In [131]:
phase2_tab

,id
0,ПС_Мангазея
1,1
2,2
3,3
4,4
...,...
604,604
605,605
606,606
607,607


In [132]:
# read tk  

tk_tab = pd.read_excel(workdir / 'clearance_grd.xlsx', sheet_name='attach', names=['id', 'set', 'x', 'y', 'z'], usecols='B,C,L,M,N')

In [133]:
tk_tab

,id,set,x,y,z
0,ПС_Мангазея,1,557359.31,7388265.89,36.47
1,ПС_Мангазея,2,557355.83,7388267.29,37.12
2,ПС_Мангазея,3,557352.48,7388268.92,36.00
3,ПС_Мангазея,4,557345.55,7388271.77,36.01
4,ПС_Мангазея,5,557342.17,7388273.08,37.21
...,...,...,...,...,...
4657,ПС_Ванкор,2,603919.55,7513480.57,53.36
4658,ПС_Ванкор,3,603916.57,7513481.75,54.18
4659,ПС_Ванкор,4,603909.03,7513484.14,54.84
4660,ПС_Ванкор,5,603905.40,7513485.15,54.17


In [134]:
from math import sqrt

def dist(a, b):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    dz = a[2] - b[2]
    
    return round(sqrt(dx*dx + dy*dy + dz*dz), 2)

In [135]:
phase2_tab.assign(l_min=np.nan, l_name=np.nan, r_min=np.nan, r_name=np.nan)


for n in range(len(phase2_tab)):
#for n in range(3):
    idx = phase2_tab.iloc[n].name
    structure = phase2_tab.iloc[n, 0]
    # водим таблицу только для выбранной опоры и только сеты не больше 10
    select = tk_tab.loc[tk_tab['id'] == structure].loc[tk_tab['set'] < 10]
    
    select_l = select.loc[select['set'] < 4]
    select_r = select.loc[select['set'] > 3].loc[select['set'] < 7]
    select_ew = select.loc[select['set'] > 6]
    #print(len(select_l), len(select_r), structure)
    
    res = []    
    for li in [select_l, select_r]:
        if len(li) > 0:
            cl_list = []
            cl_list.append((dist((li.iloc[0, 2], li.iloc[0, 3], li.iloc[0, 4]), 
                           (li.iloc[1, 2], li.iloc[1, 3], li.iloc[1, 4])), '1-2'))
            cl_list.append((dist((li.iloc[0, 2], li.iloc[0, 3], li.iloc[0, 4]), 
                           (li.iloc[2, 2], li.iloc[2, 3], li.iloc[2, 4])), '1-3'))
            cl_list.append((dist((li.iloc[1, 2], li.iloc[1, 3], li.iloc[1, 4]), 
                           (li.iloc[2, 2], li.iloc[2, 3], li.iloc[2, 4])), '2-3'))
            if len(select_ew) > 0:
                cl_list.append((dist((li.iloc[0, 2], li.iloc[0, 3], li.iloc[0, 4]), 
                                    (select_ew.iloc[0, 2], select_ew.iloc[0, 3], select_ew.iloc[0, 4])), 'ew-1'))
                cl_list.append((dist((li.iloc[1, 2], li.iloc[1, 3], li.iloc[1, 4]), 
                                    (select_ew.iloc[0, 2], select_ew.iloc[0, 3], select_ew.iloc[0, 4])), 'ew-2'))
                cl_list.append((dist((li.iloc[2, 2], li.iloc[2, 3], li.iloc[2, 4]), 
                                    (select_ew.iloc[0, 2], select_ew.iloc[0, 3], select_ew.iloc[0, 4])), 'ew-3'))
        
            res.append(sorted(cl_list)[0])
        else:
            res.append((0, 0))
        
    #print(res[0][0])
    phase2_tab.loc[idx, 'l_min'] = res[0][0]
    phase2_tab.loc[idx, 'l_name'] = res[0][1]
    phase2_tab.loc[idx, 'r_min'] = res[1][0]
    phase2_tab.loc[idx, 'r_name'] = res[1][1]


In [136]:
phase2_tab

,id,l_min,l_name,r_min,r_name
0,ПС_Мангазея,3.81,1-2,3.79,2-3
1,1,6.30,1-2,6.40,2-3
2,2,6.55,1-2,6.52,1-2
3,3,6.79,1-2,6.78,1-2
4,4,6.82,1-2,6.74,2-3
...,...,...,...,...,...
604,604,6.79,1-2,6.76,1-2
605,605,6.59,1-2,6.64,1-2
606,606,6.76,2-3,6.76,2-3
607,607,6.78,2-3,6.70,2-3


### теперь тоже самое но только тросы и вертикаль

In [137]:
for n in range(len(phase2_tab)):
#for n in range(3):
    idx = phase2_tab.iloc[n].name
    structure = phase2_tab.iloc[n, 0]

    select = tk_tab.loc[tk_tab['id'] == structure]
    
    left_z = []
    right_z = []
    ew_z = []

    left_z.extend(list(select.loc[select['set'] < 4]['z'].values))
    left_z.extend(list(select.loc[select['set'] > 10].loc[select['set'] < 14]['z'].values))
    
    right_z.extend(list(select.loc[select['set'] > 3].loc[select['set'] < 7]['z'].values))
    right_z.extend(list(select.loc[select['set'] > 13].loc[select['set'] < 17]['z'].values))
    
    ew_z.extend(list(select.loc[select['set'] > 6].loc[select['set'] < 10]['z'].values))
    ew_z.extend(list(select.loc[select['set'] > 16].loc[select['set'] < 19]['z'].values))

    #print(structure, left_z, right_z, ew_z)
    
    if len(ew_z) > 0:
        
        if len(left_z) > 0:
            l_min = round((min(ew_z) - max(left_z)), 2)
        else:
            l_min = '-'

        if len(right_z) > 0:
            r_min = round((min(ew_z) - max(right_z)), 2)
        else:
            r_min = '-'
    else:
        r_min = l_min = '-'
    
        
    
    #print(l_min, r_min)
    
    phase2_tab.loc[idx, 'ewl_vert'] = l_min
    phase2_tab.loc[idx, 'ewr_vert'] = r_min


In [138]:
phase2_tab

,id,l_min,l_name,r_min,r_name,ewl_vert,ewr_vert
0,ПС_Мангазея,3.81,1-2,3.79,2-3,5.9,5.81
1,1,6.30,1-2,6.40,2-3,8.09,8.11
2,2,6.55,1-2,6.52,1-2,9.42,9.44
3,3,6.79,1-2,6.78,1-2,8.06,8.23
4,4,6.82,1-2,6.74,2-3,8.48,8.5
...,...,...,...,...,...,...,...
604,604,6.79,1-2,6.76,1-2,8.25,8.03
605,605,6.59,1-2,6.64,1-2,8.84,8.9
606,606,6.76,2-3,6.76,2-3,8.55,8.56
607,607,6.78,2-3,6.70,2-3,8.86,8.72


### теперь середины пролетов - мин разница с тросом по вертикали

In [140]:
# read tk  again

tk_tab = pd.read_excel(workdir / 'clearance_grd.xlsx', sheet_name='attach', names=['id', 'set', 'z'], usecols='B,C,Q')

In [141]:
tk_tab

,id,set,z
0,ПС_Мангазея,1,37.55
1,ПС_Мангазея,2,40.82
2,ПС_Мангазея,3,34.50
3,ПС_Мангазея,4,34.27
4,ПС_Мангазея,5,40.98
...,...,...,...
4657,ПС_Ванкор,2,0.00
4658,ПС_Ванкор,3,0.00
4659,ПС_Ванкор,4,0.00
4660,ПС_Ванкор,5,0.00


In [142]:
for n in range(len(phase2_tab)):
#for n in range(3):
    idx = phase2_tab.iloc[n].name
    structure = phase2_tab.iloc[n, 0]

    select = tk_tab.loc[tk_tab['id'] == structure]
    if len(select) > 10:
        select = tk_tab.loc[tk_tab['id'] == structure].loc[select['set'] > 10]

   
    left_z = []
    right_z = []
    ew_z = []

    left_z.extend(list(select.loc[select['set'] < 4]['z'].values))
    left_z.extend(list(select.loc[select['set'] > 10].loc[select['set'] < 14]['z'].values))
    
    right_z.extend(list(select.loc[select['set'] > 3].loc[select['set'] < 7]['z'].values))
    right_z.extend(list(select.loc[select['set'] > 13].loc[select['set'] < 17]['z'].values))
    
    ew_z.extend(list(select.loc[select['set'] > 6].loc[select['set'] < 10]['z'].values))
    ew_z.extend(list(select.loc[select['set'] > 16].loc[select['set'] < 19]['z'].values))

    
    #print(structure, left_z, right_z, ew_z)
    
    if len(ew_z) > 0:
        
        if len(left_z) > 0:
            l_min = round((min(ew_z) - max(left_z)), 2)
        else:
            l_min = '-'

        if len(right_z) > 0:
            r_min = round((min(ew_z) - max(right_z)), 2)
        else:
            r_min = '-'
    else:
        r_min = l_min = '-'
    
        
    
    #print(l_min, r_min)
    
    phase2_tab.loc[idx, 'l_mid_vert'] = l_min
    phase2_tab.loc[idx, 'r_mid_vert'] = r_min


In [143]:
# save tab
phase2_tab.to_csv(workdir / 'phases_struct.txt', sep='\t', encoding = 'cp1251')